In [1]:
import pandas as pd
import numpy as np


In [2]:
df_application_train = pd.read_csv('application_train.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'],nrows=100000)
df_previous_application = pd.read_csv('previous_application.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'],nrows=500000)
df_installment_payments = pd.read_csv('installments_payments.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'],nrows=500000)
df_bureau = pd.read_csv('bureau.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'],nrows=500000)
df_bureau_bal = pd.read_csv('bureau_balance.csv',header=0,error_bad_lines=False,index_col=0,nrows=5000)
df_pos_cash_bal = pd.read_csv('POS_CASH_balance.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'],nrows=5000)
df_credit_card_bal = pd.read_csv('credit_card_balance.csv',header=0,error_bad_lines=False,nrows=5000)

dflist = [df_application_train,df_previous_application,df_installment_payments,df_bureau,df_bureau_bal,df_pos_cash_bal,df_credit_card_bal]


Started with Previous Application Dataset because it had historical information about the application. My main focus was on 3 noticable columns. 

1. FLAG_LAST_APPL_PER_CONTRACT: If this column had the value of 'N', then it was more than likely a mistake or duplicate of another application. My thought is that this maybe unnecessary data to the purpose of this project.

2. AMT_APPLICATION: This column states how much the applicant has asked for from the bank. 

3. AMT_CREDIT: This column states how much the bank as credited the applicant.

        My thought is if (2) and (3) are 0, then the application is incomplete and it would not provide me with additional information about the application. Thus I should remove these rows from my dataset.
        
        
In the next cell, I have worked on removing those rows from the previous_application dataset. 

In [3]:
df_previous_application = df_previous_application.drop(df_previous_application[df_previous_application['FLAG_LAST_APPL_PER_CONTRACT']=='N'].index)
df_previous_application = df_previous_application.drop(df_previous_application[(df_previous_application['AMT_APPLICATION']==0.0) & (df_previous_application['AMT_CREDIT']==0.0)].index)



In the next cell, the 'RATE_DOWN_PAYMENT','RATE_INTEREST_PRIMARY', and 'RATE_INTEREST_PRIVILEGED' columns had 'NaN' values. Because I will be aggregating the columns to merge them into the main dataset (application_train), I should replace the NaN values with 0. 

In [4]:
df_previous_application[['RATE_DOWN_PAYMENT','RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED']] = df_previous_application[['RATE_DOWN_PAYMENT','RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED']].fillna(0.0)

Below is where I aggreagated the previous_application dataset by using index 'SK_ID_CURR'. The purpose of this is so I can provide a 1:1 relationship to the main

In [5]:
df_prevapp_pivot = pd.pivot_table(df_previous_application,values=df_previous_application.columns,index='SK_ID_CURR',aggfunc={'SK_ID_PREV':(lambda x: len(x.unique())), 'NAME_CONTRACT_TYPE': (lambda x: len(x.unique())), 'AMT_ANNUITY':'sum', 'AMT_APPLICATION': 'sum',
       'AMT_CREDIT':'sum', 'AMT_DOWN_PAYMENT':'sum', 'AMT_GOODS_PRICE':'sum',
       'WEEKDAY_APPR_PROCESS_START':(lambda x: len(x.unique())), 'HOUR_APPR_PROCESS_START':np.mean,
       'FLAG_LAST_APPL_PER_CONTRACT':(lambda x: len(x.unique())), 'NFLAG_LAST_APPL_IN_DAY':(lambda x: len(x.unique())),
       'RATE_DOWN_PAYMENT':np.mean, 'RATE_INTEREST_PRIMARY':np.mean,
       'RATE_INTEREST_PRIVILEGED':np.mean, 'NAME_CASH_LOAN_PURPOSE':(lambda x: len(x.unique())),
       'NAME_CONTRACT_STATUS':(lambda x:x.map({'Approved':1,'Canceled':0,'Refused':0,'Unused offer':1}).mean()), 'DAYS_DECISION':np.mean, 'NAME_PAYMENT_TYPE':(lambda x: len(x.unique())),
       'CODE_REJECT_REASON':(lambda x: len(x.unique())), 'NAME_TYPE_SUITE':(lambda x: len(x.unique())), 'NAME_CLIENT_TYPE':(lambda x: len(x.unique())),
       'NAME_GOODS_CATEGORY':(lambda x: len(x.unique())), 'NAME_PORTFOLIO':(lambda x: len(x.unique())), 'NAME_PRODUCT_TYPE':(lambda x: len(x.unique())),
       'CHANNEL_TYPE':(lambda x: len(x.unique())), 'SELLERPLACE_AREA':(lambda x: len(x.unique())), 'NAME_SELLER_INDUSTRY':(lambda x: len(x.unique())),
       'CNT_PAYMENT':'sum', 'NAME_YIELD_GROUP':(lambda x: len(x.unique())), 'PRODUCT_COMBINATION':(lambda x: len(x.unique())),
       'DAYS_FIRST_DRAWING':'sum', 'DAYS_FIRST_DUE':'sum', 'DAYS_LAST_DUE_1ST_VERSION':'sum',
       'DAYS_LAST_DUE':'sum', 'DAYS_TERMINATION':'sum', 'NFLAG_INSURED_ON_APPROVAL':np.mean})

In [6]:
df_prevapp_pivot.head()

,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,CHANNEL_TYPE,CNT_PAYMENT,CODE_REJECT_REASON,DAYS_DECISION,DAYS_FIRST_DRAWING,...,NAME_YIELD_GROUP,NFLAG_INSURED_ON_APPROVAL,NFLAG_LAST_APPL_IN_DAY,PRODUCT_COMBINATION,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,SELLERPLACE_AREA,SK_ID_PREV,WEEKDAY_APPR_PROCESS_START
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,24835.5,23787.0,2520.0,24835.5,1,8.0,1,-1740.0,365243.0,...,1,0.0,1,1,0.104326,0.0,0.0,1,1,1
100006,24246.000,675000.0,675000.0,0.0,675000.0,1,48.0,1,-181.0,365243.0,...,1,0.0,1,1,0.000000,0.0,0.0,1,1,1
100007,29520.585,405000.0,464400.0,0.0,405000.0,2,66.0,1,-866.0,365243.0,...,2,0.0,1,2,0.000000,0.0,0.0,2,2,2
100008,17885.835,162598.5,162598.5,0.0,162598.5,1,10.0,1,-370.0,365243.0,...,1,0.0,1,1,0.000000,0.0,0.0,1,1,1
100009,8996.760,98239.5,98239.5,0.0,98239.5,1,12.0,1,-449.0,365243.0,...,1,0.0,1,1,0.000000,0.0,0.0,1,1,1


# Merging previous_application dataset into application_train dataset 

In [7]:


df = df_application_train.merge(df_prevapp_pivot,on =['SK_ID_CURR'],suffixes=('_app_train','_prev_app'),how='left')

# Note the odd data within the columns

Days_Employed: 365243 default value



In [8]:
print(df.describe())

              TARGET   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT_app_train  \
count  100000.000000  100000.000000      1.000000e+05          1.000000e+05   
mean        0.080930       0.417390      1.694261e+05          5.990034e+05   
std         0.272729       0.721014      3.835007e+05          4.020520e+05   
min         0.000000       0.000000      2.565000e+04          4.500000e+04   
25%         0.000000       0.000000      1.125000e+05          2.700000e+05   
50%         0.000000       0.000000      1.440000e+05          5.130405e+05   
75%         0.000000       1.000000      2.025000e+05          8.086500e+05   
max         1.000000      12.000000      1.170000e+08          4.050000e+06   

       AMT_ANNUITY_app_train  AMT_GOODS_PRICE_app_train  \
count           99993.000000               9.991900e+04   
mean            27085.325928               5.383451e+05   
std             14459.617534               3.693977e+05   
min              1980.000000               4.500000e

# Aggreagated Bureau & Installment datasets and then merged into dataset

In [9]:
df_bureau_pivot = pd.pivot_table(df_bureau,values=df_bureau.columns,index='SK_ID_CURR',aggfunc={'SK_ID_BUREAU':'count','CREDIT_ACTIVE':'count','CREDIT_CURRENCY':(lambda x: len(x.unique())),'DAYS_CREDIT':np.min,'CREDIT_DAY_OVERDUE':'sum','DAYS_CREDIT_ENDDATE':'sum', 'DAYS_ENDDATE_FACT':'sum', 'AMT_CREDIT_MAX_OVERDUE':'sum', 'CNT_CREDIT_PROLONG':'sum', 'AMT_CREDIT_SUM':'sum','AMT_CREDIT_SUM_DEBT':'sum', 'AMT_CREDIT_SUM_LIMIT':'sum', 'AMT_CREDIT_SUM_OVERDUE':'sum',
       'CREDIT_TYPE':(lambda x: len(x.unique())), 'DAYS_CREDIT_UPDATE':np.min, 'AMT_ANNUITY':'sum'})

In [10]:
df = df.merge(df_bureau_pivot,on =['SK_ID_CURR'],suffixes=('_curr','_bur'),how='left')

In [11]:
df_installment_payments_pivot = pd.pivot_table(df_installment_payments,values=df_installment_payments.columns,index='SK_ID_CURR',aggfunc={'SK_ID_PREV':(lambda x: len(x.unique())),'NUM_INSTALMENT_VERSION':'sum', 'NUM_INSTALMENT_NUMBER':'sum','DAYS_INSTALMENT':'sum', 'DAYS_ENTRY_PAYMENT':'sum', 'AMT_INSTALMENT':'sum','AMT_PAYMENT':'sum'})                                                                                                                      

# Added Columns to show differences before merging


AMT_INSTAL_PAY_DIFF: I believe that knowing how much the applicant has paid back in the past would be a good indicator for the future

NUM_INSTAL_VERSION_NUM_DIFF: I believe knowing how fast or slow the previous load was paid would also be a good indicator.

DAY_INSTAL_ENTRY_DIFF: Similar to NUM_INSTAL_VERSION_NUM_DIFF

In [12]:
df_installment_payments_pivot['AMT_INSTAL_PAY_DIFF'] = df_installment_payments_pivot['AMT_INSTALMENT']-df_installment_payments_pivot['AMT_PAYMENT']
df_installment_payments_pivot['NUM_INSTAL_VERSION_NUM_DIFF'] = df_installment_payments_pivot['NUM_INSTALMENT_VERSION']-df_installment_payments_pivot['NUM_INSTALMENT_NUMBER']
df_installment_payments_pivot['DAY_INSTAL_ENTRY_DIFF'] = df_installment_payments_pivot['DAYS_INSTALMENT']-df_installment_payments_pivot['DAYS_ENTRY_PAYMENT']


In [13]:
df = df.merge(df_installment_payments_pivot,on =['SK_ID_CURR'],suffixes=('_curr2','_instpay'),how='left')

In [22]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 100002 to 216090
Columns: 184 entries, TARGET to BANKED
dtypes: bool(1), float64(127), int64(40), object(16)
memory usage: 143.0+ MB
None


In [15]:
df['BANKED']= df.index.isin(df_bureau.index)